In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
sns.set_theme(style="darkgrid")

#Word2Vec
from langdetect import detect
import nltk
from nltk.tokenize import word_tokenize
import gensim
from gensim.models import Word2Vec

#keras
import tensorflow as tf
import keras

Using TensorFlow backend.


In [2]:
%%bash
ls

LiteratureReport.pdf
main.ipynb
ml.ipynb
OLID
README.md
roberta.large
script.ipynb
second.ipynb
SOLID


In [3]:
# Load datasets and rename columns, load all aqi data but specify metro data name
def loadcsv():
    train = pd.read_csv('OLID/olid-training-v1.0.tsv',delimiter='\t',skiprows=1,header=None)
#     train.drop([3], axis = 1,inplace=True)
#     train.drop([4], axis = 1,inplace=True)
    train[2] = np.where(train[2]=="NOT",0,1)
    
    testlabel = pd.read_csv('OLID/labels-levela.csv',delimiter=',',header=None,dtype=str)
    testlabel.drop([0], axis = 1,inplace=True)
    testlabel.rename(columns={1: 2},inplace=True)
    
    testdata = pd.read_csv('OLID/testset-levela.tsv',delimiter='\t')
    print(testdata.columns) 
    
    test = pd.concat([testdata,testlabel], axis=1)
    test[2] = np.where(test[2]=="NOT",0,1)
    test.rename(columns={'id':0,'tweet':1},inplace=True)
    
    return train,test
train,test=loadcsv()

Index(['id', 'tweet'], dtype='object')


In [4]:
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())

    ## Tokenize (convert from string to list)
    lst_text = text.split()    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in lst_stopwords]

    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]

    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]

    ## back to string from list
    text = " ".join(lst_text)
    return text
def processdf(df):
    df['tweet'] = df[1].apply(utils_preprocess_text)
    df['tweet'] = df['tweet'].apply(word_tokenize)
processdf(train)
processdf(test)
test

,0,1,2,tweet
0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,1,"[whoisq, wherestheserver, dumpnike, declasfisa..."
1,27014,"#ConstitutionDay is revered by Conservatives, ...",0,"[constitutionday, is, revered, by, conservativ..."
2,30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,0,"[foxnews, nra, maga, potus, trump, 2ndamendmen..."
3,13876,#Watching #Boomer getting the news that she is...,0,"[watching, boomer, getting, the, news, that, s..."
4,60133,#NoPasaran: Unity demo to oppose the far-right...,1,"[nopasaran, unity, demo, to, oppose, the, farr..."
...,...,...,...,...
855,73439,#DespicableDems lie again about rifles. Dem Di...,1,"[despicabledems, lie, again, about, rifle, dem..."
856,25657,#MeetTheSpeakers 🙌 @USER will present in our e...,0,"[meetthespeakers, user, will, present, in, our..."
857,67018,3 people just unfollowed me for talking about ...,1,"[3, people, just, unfollowed, me, for, talking..."
858,50665,#WednesdayWisdom Antifa calls the right fascis...,0,"[wednesdaywisdom, antifa, call, the, right, fa..."


In [27]:
def getEmbeddings(train,test):
    corpus = pd.concat([train['tweet'],test['tweet']])
    # print(len(train),len(test),len(corpus))
    vecmodel = gensim.models.word2vec.Word2Vec(corpus, size=300,window=8, min_count=1, sg=1, iter=30)
    tk=tf.keras.preprocessing.text.Tokenizer(lower=True,split=' ',oov_token="NaN", filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
    tk.fit_on_texts(corpus)

    train['tweet_id'] = tk.texts_to_sequences(train['tweet'])
    test['tweet_id'] = tk.texts_to_sequences(test['tweet'])

    Xtrain = tf.keras.preprocessing.sequence.pad_sequences(train['tweet_id'], maxlen=15,padding="post", truncating="post",value=0)
    Xtest = tf.keras.preprocessing.sequence.pad_sequences(test['tweet_id'], maxlen=15,padding="post", truncating="post",value=0)

    ytrain = np.array(train[2]).reshape((train[2].shape[0],1))
    ytest = np.array(test[2]).reshape((test[2].shape[0],1))
    
    print(vecmodel.wv['she'])
    embed = np.zeros((len(tk.word_index)+1, 300))
    for word,idx in tk.word_index.items():
        try:
            embed[idx]=vecmodel.wv[word]
        except:
            pass
        
    print(Xtrain.shape,Xtest.shape)
    print(ytrain.shape,ytest.shape)
    print(embed.shape)
    
    return Xtrain,Xtest,ytrain,ytest,embed

Xtrain,Xtest,ytrain,ytest,embed = getEmbeddings(train,test)

[-7.28740916e-02  4.39802200e-01 -4.63868052e-01 -1.70535102e-01
 -4.23907429e-01  3.06187987e-01  1.12206675e-01 -1.81175873e-01
  2.23047078e-01  3.91755819e-01  4.42473404e-02  4.26364802e-02
  4.27939594e-01 -1.74144566e-01  7.30657652e-02 -2.58322299e-01
 -2.65732318e-01 -2.86689013e-01  3.62401247e-01 -1.90714255e-01
  8.62595066e-02  4.37412709e-01 -1.75225988e-01  1.01692580e-01
  3.92160825e-02 -1.35381728e-01 -3.28089505e-01  2.71916300e-01
  2.28904709e-01  1.47348158e-02  2.32596487e-01 -2.26412583e-02
  7.68175796e-02 -3.07510704e-01  2.74040550e-01  2.47283921e-01
  3.43205094e-01  1.22091353e-01 -3.36508870e-01  9.48349237e-02
 -1.57745302e-01 -2.60672003e-01 -1.51178285e-01 -3.72529119e-01
  1.78581104e-01 -1.85131598e-02  3.40127438e-01 -3.26425172e-02
 -4.16175753e-01  3.01316917e-01  8.88589621e-02 -1.13395274e-01
 -4.35705990e-01 -5.92932642e-01 -3.26424211e-01  1.96147189e-01
 -5.31980433e-02  1.73851311e-01 -1.21057726e-01 -1.78394094e-01
 -1.06220655e-01 -1.89827

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from keras.layers import Embedding
from keras.layers import Bidirectional
from keras.layers import Dense
from keras.layers import Flatten

def attention_layer(inputs, neurons):
    x = keras.layers.Permute((2,1))(inputs)
    x = keras.layers.Dense(neurons, activation="softmax")(x)
    x = keras.layers.Permute((2,1), name="attention")(x)
    x = keras.layers.multiply([inputs, x])
    return x
with tf.device('/CPU:0'):
    x_in = keras.layers.Input(shape=(15,))
    x = Embedding(input_dim=embed.shape[0],output_dim=embed.shape[1],weights=[embed],input_length=15,trainable=False)(x_in)
    x = attention_layer(x, neurons=15)
    x = Bidirectional(keras.layers.LSTM(units=15, dropout=0.2, return_sequences=True))(x)
    x = Bidirectional(keras.layers.LSTM(units=15, dropout=0.2))(x)
    x = Dense(64, activation='relu')(x)
    y_out = Dense(1, activation='sigmoid')(x)
    model = keras.models.Model(x_in, y_out)
    model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
    print(model.summary())

    training = model.fit(Xtrain,ytrain,batch_size=256,epochs=10, shuffle=True, verbose=1, validation_split=0.2)

    testPred = model.predict(Xtest)
    trainPred = model.predict(Xtrain)

# print(mean_squared_error(testPred, ytest,squared=False))
# print(mean_squared_error(trainPred, ytrain,squared=False))
# print(mean_absolute_error(testPred, ytest))
# print(mean_absolute_error(trainPred, ytrain))

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 15)           0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 15, 300)      6131400     input_4[0][0]                    
__________________________________________________________________________________________________
permute_4 (Permute)             (None, 300, 15)      0           embedding_4[0][0]                
__________________________________________________________________________________________________
dense_10 (Dense)                (None, 300, 15)      240         permute_4[0][0]                  
____________________________________________________________________________________________

In [ ]:
testPred

In [ ]:
metrics = [k for k in training.history.keys() if ("loss" not in k) and ("val" not in k)]
fig, ax = plt.subplots(nrows=1, ncols=2, sharey=True)
ax[0].set(title="Training")
ax11 = ax[0].twinx()
ax[0].plot(training.history['loss'], color='black')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Loss', color='black')
for metric in metrics:
    ax11.plot(training.history[metric], label=metric)
ax11.set_ylabel("Score", color='steelblue')
ax11.legend()
ax[1].set(title="Validation")
ax22 = ax[1].twinx()
ax[1].plot(training.history['val_loss'], color='black')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Loss', color='black')
for metric in metrics:
     ax22.plot(training.history['val_'+metric], label=metric)
ax22.set_ylabel("Score", color="steelblue")
plt.show()